
<p align="left">
      <img src="oft.png" 
       width="300">
<br>

# Modern Portfolio Theory

### Notebook by [Marco Tavora](https://marcotavora.me/)

## Table of contents

1. [Goal](#Goal)
2. [Data Handling](#Data-Handling)
3. [Covariances](#Covariances)

## Goal
[[go back to the top]](#Table-of-contents)

The goal of this notebook is to build optimal modern portfolio theory (MPT) portfolios based on the Sharpe ratio. We will start with the three stock prices/indexes below.

In [56]:
import pandas as pd
import aux_1 as af
import warnings      
warnings.filterwarnings('ignore')

In [31]:
files = ['smlv_prices.csv', 
         'lng_prices.csv', 
         'spy_prices.csv']


smlv_prices = pd.read_csv(files[0])
lng_prices = pd.read_csv(files[1])
spy_prices = pd.read_csv(files[2])

data = [smlv_prices, lng_prices, spy_prices]
new_data = []
for el in data:
    el.index = el['Date']
    aux = el[['Close']]
    new_data.append(aux)
    

In [32]:
[smlv_prices, lng_prices, spy_prices] = [new_data[0], 
                                         new_data[1], 
                                         new_data[2]]

[smlv_prices.columns, lng_prices.columns, spy_prices.columns] = [['smlv'], 
                                                                  ['lng'], 
                                                                  ['spy']]

smlv_prices.head()
lng_prices.head()
spy_prices.head()

,smlv
Date,
2013-02-21,59.570000
2013-02-22,60.049999
2013-02-25,59.520000
2013-02-26,59.230000
2013-02-27,59.500000


,lng
Date,
1994-04-04,24.0
1994-04-05,36.0
1994-04-06,36.0
1994-04-07,36.0
1994-04-08,36.0


,spy
Date,
1994-04-04,43.9062
1994-04-05,44.8125
1994-04-06,44.8125
1994-04-07,45.0312
1994-04-08,44.6875


## Data Handling
[[go back to the top]](#Table-of-contents)

We first join the `Close` columns from the three `DataFrames`, clean-up `NaNs` and convert the indices into `datetime` format. We can then quickly calculate the daily and monthly returns:

In [53]:
daily = pd.concat([smlv_prices, lng_prices, spy_prices], axis=1).dropna()
daily.index = pd.to_datetime(daily.index)
daily_returns = daily.pct_change()
daily_returns.dropna(inplace=True)
print('Daily prices:')
daily.head()
print('Daily Returns:')
daily_returns.head()

Daily prices:


,smlv,lng,spy
2013-02-21,59.570000,20.209999,150.419998
2013-02-22,60.049999,20.990000,151.889999
2013-02-25,59.520000,20.440001,149.000000
2013-02-26,59.230000,21.150000,150.020004
2013-02-27,59.500000,21.570000,151.910004


Daily Returns:


,smlv,lng,spy
2013-02-22,0.008058,0.038595,0.009773
2013-02-25,-0.008826,-0.026203,-0.019027
2013-02-26,-0.004872,0.034736,0.006846
2013-02-27,0.004559,0.019858,0.012598
2013-02-28,0.008571,-0.012517,-0.001975


We can resample the daily data into `business month start frequency` data and then calculate the $\%$ change:

In [54]:
monthly = daily.resample('BMS').first()
monthly_returns = monthly.pct_change()
monthly_returns.dropna(inplace=True)
print('Monthly prices:')
monthly.head()
print('Monthly returns:')
monthly_returns.tail()

Monthly prices:


,smlv,lng,spy
2013-02-01,59.570000,20.209999,150.419998
2013-03-01,59.599998,21.160000,152.110001
2013-04-01,61.689999,27.940001,156.050003
2013-05-01,61.980000,27.889999,158.279999
2013-06-03,63.230000,29.240000,164.350006


Monthly returns:


,smlv,lng,spy
2018-06-01,0.050541,0.123787,0.032531
2018-07-02,0.012213,-0.027121,-0.006360
2018-08-01,0.009128,-0.021803,0.033105
2018-09-03,0.032439,0.069257,0.031866
2018-10-01,-0.034643,0.045563,0.006625


## Covariance
[[go back to the top]](#Table-of-contents)

Risk can be measured by the volatility. More concretely, the portfolio return variance is given by:

$$\sigma _p^2 = \sum\limits_i {w_i^2} \sigma _i^2 + \sum\limits_i {\sum\limits_{j \ne i} {{w_i}} } {w_j}{\sigma _i}{\sigma _j}{\rho _{ij}}$$

We first evaluate the covariance matrix of the daily returns:

In [70]:
covariances = {}
for i in monthly_returns.index:
    covariances[i] = daily_returns[(daily_returns.index.month == i.month) & 
                                   (daily_returns.index.year == i.year)].cov()
covariances[i]

,smlv,lng,spy
smlv,0.000072,0.000055,0.000066
lng,0.000055,0.000180,0.000123
spy,0.000066,0.000123,0.000125
